manually install torch with cuda:
```terminal
pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
```

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as T
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.amp import autocast, GradScaler
import torch.nn.functional as F
import copy
import numpy as np
import optuna
import time
import itertools
import pandas as pd

import optuna

c:\Users\Jax\Coding\Optimierungsverfahren\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### PyTorch with CUDA

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dtype = torch.float32
print("Using device:", device)

Using device: cuda:0


### Get CIFAR10 data

In [3]:
TRAINING_SIZE = 49000 # full data
# TRAINING_SIZE = 5000    # develop data

VALIDATION_SIZE = 1000
BATCH_SIZE = 512
NUM_WORKERS = 12

iters_per_epoch = np.ceil(TRAINING_SIZE / BATCH_SIZE)

RANDOM_SEED = 0
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [4]:
transform_train = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomCrop(32, padding=4),
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_val_test = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

cifar10_train = dset.CIFAR10('datasets', train=True, download=True, transform=transform_train)
# cifar10_val = dset.CIFAR10('datasets', train=True, download=True, transform=transform_val_test)
cifar10_test = dset.CIFAR10('datasets', train=False, download=True, transform=transform_val_test)

indices = np.arange(len(cifar10_train))
np.random.shuffle(indices)
train_idx = indices[:TRAINING_SIZE]
val_idx = indices[TRAINING_SIZE:TRAINING_SIZE + VALIDATION_SIZE]

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

loader_train = DataLoader(
    cifar10_train,
    batch_size=BATCH_SIZE,
    sampler=train_sampler,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    persistent_workers=True
)

loader_val = DataLoader(
    cifar10_train,
    batch_size=BATCH_SIZE,
    sampler=val_sampler,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    persistent_workers=True
)

print(f"Train batches: {len(loader_train)}, Val batches: {len(loader_val)}")
print(f"Results in {iters_per_epoch} iterations per epoch.")

Train batches: 96, Val batches: 2
Results in 96.0 iterations per epoch.


### Model and Hyperparameter Training

In [5]:
def train_model(
    model,
    train_loader,
    val_loader,
    optimizer,
    num_epochs=10,
    criterion=None,
    device=None,
    print_every=20,
    patience=3,
    accumulation_steps=1,
    scheduler=None
):
    """
    Optimized training function with AMP, early stopping, gradient accumulation, 
    best model saving, and optional scheduler.
    """
    device = device or torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    criterion = criterion or F.cross_entropy
    optimizer = optimizer
    scaler = GradScaler()

    best_val_acc = -float('inf')
    best_model = None
    epochs_no_improve = 0

    train_accs, val_accs, lrs, epoch_times = [], [], [], []
    start_time = time.time()

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad() if accumulation_steps == 1 else None

            with autocast("cuda"):
                outputs = model(x)
                loss = criterion(outputs, y) / accumulation_steps  # scale loss for accumulation

            scaler.scale(loss).backward()

            if (i + 1) % accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            
            running_loss += loss.item() * accumulation_steps
            if (i + 1) % print_every == 0:
                print(f"Epoch {epoch+1}, Iter {i+1}, Avg Loss: {running_loss/print_every:.4f}")
                running_loss = 0.0

        # Step scheduler at epoch end
        if scheduler is not None:
            scheduler.step()
            lrs.append(scheduler.get_last_lr()[0])

        # Validation check
        val_acc = get_accuracy(val_loader, model, f"Validation Epoch {epoch+1}", device)
        val_accs.append(val_acc)
        
        train_acc = get_accuracy(train_loader, model, f"Training Epoch {epoch+1}", device=device)
        train_accs.append(train_acc)

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = copy.deepcopy(model)
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs.")
                break

    total_time = time.time() - start_time

    return {
        "model": best_model if best_model else model,
        "train_accs": train_accs,
        "val_accs": val_accs,
        "lrs": lrs,
        "epoch_times": epoch_times,
        "total_time": total_time
    }


def get_accuracy(loader, model, name="Validation", device=device, debug=True):
    """
    Compute accuracy of the model on a given DataLoader.
    """
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            with autocast("cuda"):
                scores = model(x)
            preds = scores.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    acc = correct / total
    if debug:
        print(f"{name} Accuracy: {100*acc:.2f}%")
    return acc

In [6]:
def hyperparam_training(learn_rate, epochs):

    ch1 = 16
    ch2 = 64
    ch3 = 32
    model = nn.Sequential(
        nn.Conv2d(3, ch1, 5, padding=2),
        nn.BatchNorm2d(ch1),
        nn.ReLU(),

        nn.Conv2d(ch1, ch2, 3, padding=1),
        nn.BatchNorm2d(ch2),
        nn.ReLU(),
        nn.MaxPool2d(2,2),

        nn.Conv2d(ch2, ch3, 3, padding=1),
        nn.BatchNorm2d(ch3),
        nn.ReLU(),
        nn.MaxPool2d(2,2),

        nn.Flatten(),
        nn.Linear(ch3 * 8 * 8, 128),
        nn.ReLU(),
        nn.Dropout(0.25),
        nn.Linear(128, 10)
    ).to(device)

    weight_decay = 2.075612963823467e-06
    optimizer = torch.optim.AdamW(model.parameters(), lr=learn_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    training_result = train_model(
        model=model,
        train_loader=loader_train,
        val_loader=loader_val,
        num_epochs=epochs,
        device=device,
        print_every=len(loader_train) // 4,
        patience=2,
        optimizer=optimizer,
        scheduler=scheduler,
    )
    trained_model = training_result["model"]
    
    val_acc = get_accuracy(loader_val, trained_model, device=device, debug=False)

    return {
        "lr": learn_rate,
        "epochs": epochs,
        "val_acc": val_acc,
        "train_accs": training_result["train_accs"],
        "val_accs": training_result["val_accs"],
        "runtime": training_result["total_time"],
    }

## Search Algorithms

In [7]:
NUM_TRIALS = 25
NUM_PER_DIM = np.sqrt(NUM_TRIALS).astype(int)
LR_MIN = 1e-6
LR_MAX = 1e-2
EPOCHS_MIN = 5
EPOCHS_MAX = 25

print(f"{NUM_TRIALS} trials will be run, {NUM_PER_DIM} per dimension.")

25 trials will be run, 5 per dimension.


### Hidden Results

In [8]:
def run_hidden_results():
    HIDDEN_TRIALS = 200
    HIDDEN_TESTS_PER_DIM = np.sqrt(HIDDEN_TRIALS).astype(int)

    lr_values = np.logspace(np.log10(LR_MIN), np.log10(LR_MAX), HIDDEN_TESTS_PER_DIM)
    epochs_values = np.linspace(EPOCHS_MIN, EPOCHS_MAX, HIDDEN_TESTS_PER_DIM, dtype=int)

    results = []
    trial = 1
    for lr, ep in itertools.product(lr_values, epochs_values):
        print(f"\n[Hidden {trial}/{HIDDEN_TRIALS}] lr={lr:.3e}, epochs={ep}")
        res = hyperparam_training(lr, ep)
        res["trial"] = trial
        res["method"] = "Hidden Search"
        results.append(res)
        trial += 1

    return pd.DataFrame(results)

### Grid Search

In [9]:
def run_grid_search():
    lr_values = np.logspace(np.log10(LR_MIN), np.log10(LR_MAX), NUM_PER_DIM)
    epochs_values = np.linspace(EPOCHS_MIN, EPOCHS_MAX, NUM_PER_DIM, dtype=int)

    results = []
    trial = 1
    for lr, ep in itertools.product(lr_values, epochs_values):
        print(f"\n[Grid {trial}/{NUM_TRIALS}] lr={lr:.3e}, epochs={ep}")
        res = hyperparam_training(lr, ep)
        res["trial"] = trial
        res["method"] = "Grid Search"
        results.append(res)
        trial += 1

    return pd.DataFrame(results)

### Random Search

In [10]:
def run_random_search():
    lr_values = 10 ** np.random.uniform(np.log10(LR_MIN), np.log10(LR_MAX), NUM_TRIALS)
    epochs_values = np.random.randint(EPOCHS_MIN, EPOCHS_MAX+1, NUM_TRIALS)

    results = []
    trial = 1
    for lr, ep in zip(lr_values, epochs_values):
        print(f"\n[Random {trial}/{NUM_TRIALS}] lr={lr:.3e}, epochs={ep}")
        res = hyperparam_training(lr, ep)
        res["trial"] = trial
        res["method"] = "Random Search"
        results.append(res)
        trial += 1
        
    return pd.DataFrame(results)

### Bayesian Search

In [11]:
def objective(trial):
    lr = trial.suggest_float("learn_rate", LR_MIN, LR_MAX, log=True)
    epochs = trial.suggest_int("epochs", EPOCHS_MIN, EPOCHS_MAX)

    results = hyperparam_training(lr, epochs)
    return results["val_acc"]

In [12]:
def run_bayesian_search():
    study_name = "bayesian_optimization"
    storage_name = f"sqlite:///{study_name}.db"

    # Reset study
    summaries = optuna.study.get_all_study_summaries(storage=storage_name)
    if study_name in [s.study_name for s in summaries]:
        optuna.delete_study(study_name=study_name, storage=storage_name)
        print(f"Deleted existing study: {study_name}")

    study = optuna.create_study(
        study_name=study_name,
        storage=storage_name,
        direction="maximize",
    )

    study.optimize(objective, n_trials=NUM_TRIALS)

    rows = []
    for t in study.trials:
        row = {
            "trial": t.number + 1,
            "lr": t.params.get("learn_rate", None),
            "epochs": t.params.get("epochs", None),
            "val_acc": t.value,
            "method": "Bayesian Search"
        }
        # todo: get time for each trial.
        rows.append(row)

    return pd.DataFrame(rows)

## Results

In [13]:
df_hidden = run_hidden_results()
df_hidden.to_csv("results/hidden.csv")


[Hidden 1/200] lr=1.000e-06, epochs=5
Epoch 1, Iter 24, Avg Loss: 2.3405
Epoch 1, Iter 48, Avg Loss: 2.3323
Epoch 1, Iter 72, Avg Loss: 2.3224
Epoch 1, Iter 96, Avg Loss: 2.3216
Validation Epoch 1 Accuracy: 11.50%
Training Epoch 1 Accuracy: 10.71%
Epoch 2, Iter 24, Avg Loss: 2.3128
Epoch 2, Iter 48, Avg Loss: 2.3094
Epoch 2, Iter 72, Avg Loss: 2.3088
Epoch 2, Iter 96, Avg Loss: 2.3052
Validation Epoch 2 Accuracy: 12.40%
Training Epoch 2 Accuracy: 12.13%
Epoch 3, Iter 24, Avg Loss: 2.2988
Epoch 3, Iter 48, Avg Loss: 2.2961
Epoch 3, Iter 72, Avg Loss: 2.2937
Epoch 3, Iter 96, Avg Loss: 2.2925
Validation Epoch 3 Accuracy: 14.50%
Training Epoch 3 Accuracy: 13.28%
Epoch 4, Iter 24, Avg Loss: 2.2883
Epoch 4, Iter 48, Avg Loss: 2.2884
Epoch 4, Iter 72, Avg Loss: 2.2855
Epoch 4, Iter 96, Avg Loss: 2.2880
Validation Epoch 4 Accuracy: 13.20%
Training Epoch 4 Accuracy: 13.50%
Epoch 5, Iter 24, Avg Loss: 2.2839
Epoch 5, Iter 48, Avg Loss: 2.2872
Epoch 5, Iter 72, Avg Loss: 2.2847
Epoch 5, Iter 96

In [14]:
df_grid = run_grid_search()
df_grid.to_csv("results/grid.csv")


[Grid 1/25] lr=1.000e-06, epochs=5
Epoch 1, Iter 24, Avg Loss: 2.3355
Epoch 1, Iter 48, Avg Loss: 2.3237
Epoch 1, Iter 72, Avg Loss: 2.3200
Epoch 1, Iter 96, Avg Loss: 2.3127
Validation Epoch 1 Accuracy: 9.70%
Training Epoch 1 Accuracy: 10.94%
Epoch 2, Iter 24, Avg Loss: 2.3094
Epoch 2, Iter 48, Avg Loss: 2.3009
Epoch 2, Iter 72, Avg Loss: 2.2994
Epoch 2, Iter 96, Avg Loss: 2.2971
Validation Epoch 2 Accuracy: 11.30%
Training Epoch 2 Accuracy: 12.09%
Epoch 3, Iter 24, Avg Loss: 2.2968
Epoch 3, Iter 48, Avg Loss: 2.2894
Epoch 3, Iter 72, Avg Loss: 2.2870
Epoch 3, Iter 96, Avg Loss: 2.2858
Validation Epoch 3 Accuracy: 12.10%
Training Epoch 3 Accuracy: 13.41%
Epoch 4, Iter 24, Avg Loss: 2.2862
Epoch 4, Iter 48, Avg Loss: 2.2816
Epoch 4, Iter 72, Avg Loss: 2.2804
Epoch 4, Iter 96, Avg Loss: 2.2802
Validation Epoch 4 Accuracy: 15.20%
Training Epoch 4 Accuracy: 14.20%
Epoch 5, Iter 24, Avg Loss: 2.2796
Epoch 5, Iter 48, Avg Loss: 2.2790
Epoch 5, Iter 72, Avg Loss: 2.2767
Epoch 5, Iter 96, Av

In [15]:
df_random = run_random_search()
df_random.to_csv("results/random.csv")


[Random 1/25] lr=4.157e-04, epochs=11
Epoch 1, Iter 24, Avg Loss: 2.0453
Epoch 1, Iter 48, Avg Loss: 1.7259
Epoch 1, Iter 72, Avg Loss: 1.6049
Epoch 1, Iter 96, Avg Loss: 1.5073
Validation Epoch 1 Accuracy: 44.40%
Training Epoch 1 Accuracy: 48.04%
Epoch 2, Iter 24, Avg Loss: 1.4467
Epoch 2, Iter 48, Avg Loss: 1.4080
Epoch 2, Iter 72, Avg Loss: 1.3766
Epoch 2, Iter 96, Avg Loss: 1.3464
Validation Epoch 2 Accuracy: 53.20%
Training Epoch 2 Accuracy: 54.22%
Epoch 3, Iter 24, Avg Loss: 1.2912
Epoch 3, Iter 48, Avg Loss: 1.2672
Epoch 3, Iter 72, Avg Loss: 1.2609
Epoch 3, Iter 96, Avg Loss: 1.2258
Validation Epoch 3 Accuracy: 56.00%
Training Epoch 3 Accuracy: 56.84%
Epoch 4, Iter 24, Avg Loss: 1.1893
Epoch 4, Iter 48, Avg Loss: 1.1723
Epoch 4, Iter 72, Avg Loss: 1.1691
Epoch 4, Iter 96, Avg Loss: 1.1674
Validation Epoch 4 Accuracy: 61.70%
Training Epoch 4 Accuracy: 60.34%
Epoch 5, Iter 24, Avg Loss: 1.1373
Epoch 5, Iter 48, Avg Loss: 1.1288
Epoch 5, Iter 72, Avg Loss: 1.1079
Epoch 5, Iter 96

In [16]:
df_bayesian = run_bayesian_search()
df_bayesian.to_csv("results/bayesian.csv")

[I 2025-12-01 19:09:50,705] A new study created in RDB with name: bayesian_optimization


Deleted existing study: bayesian_optimization
Epoch 1, Iter 24, Avg Loss: 2.2355
Epoch 1, Iter 48, Avg Loss: 2.0814
Epoch 1, Iter 72, Avg Loss: 1.9540
Epoch 1, Iter 96, Avg Loss: 1.8757
Validation Epoch 1 Accuracy: 35.50%
Training Epoch 1 Accuracy: 36.47%
Epoch 2, Iter 24, Avg Loss: 1.7950
Epoch 2, Iter 48, Avg Loss: 1.7455
Epoch 2, Iter 72, Avg Loss: 1.7021
Epoch 2, Iter 96, Avg Loss: 1.6700
Validation Epoch 2 Accuracy: 41.40%
Training Epoch 2 Accuracy: 43.12%
Epoch 3, Iter 24, Avg Loss: 1.6190
Epoch 3, Iter 48, Avg Loss: 1.5989
Epoch 3, Iter 72, Avg Loss: 1.5745
Epoch 3, Iter 96, Avg Loss: 1.5564
Validation Epoch 3 Accuracy: 45.90%
Training Epoch 3 Accuracy: 46.44%
Epoch 4, Iter 24, Avg Loss: 1.5282
Epoch 4, Iter 48, Avg Loss: 1.5130
Epoch 4, Iter 72, Avg Loss: 1.4952
Epoch 4, Iter 96, Avg Loss: 1.4814
Validation Epoch 4 Accuracy: 49.30%
Training Epoch 4 Accuracy: 49.17%
Epoch 5, Iter 24, Avg Loss: 1.4683
Epoch 5, Iter 48, Avg Loss: 1.4554
Epoch 5, Iter 72, Avg Loss: 1.4455
Epoch 5, 

[I 2025-12-01 19:10:42,296] Trial 0 finished with value: 0.594 and parameters: {'learn_rate': 6.070770490926786e-05, 'epochs': 25}. Best is trial 0 with value: 0.594.


Training Epoch 17 Accuracy: 59.85%
Early stopping triggered after 17 epochs.
Epoch 1, Iter 24, Avg Loss: 2.0380
Epoch 1, Iter 48, Avg Loss: 1.7707
Epoch 1, Iter 72, Avg Loss: 1.6136
Epoch 1, Iter 96, Avg Loss: 1.5247
Validation Epoch 1 Accuracy: 43.50%
Training Epoch 1 Accuracy: 43.82%
Epoch 2, Iter 24, Avg Loss: 1.4710
Epoch 2, Iter 48, Avg Loss: 1.4163
Epoch 2, Iter 72, Avg Loss: 1.3808
Epoch 2, Iter 96, Avg Loss: 1.3504
Validation Epoch 2 Accuracy: 53.10%
Training Epoch 2 Accuracy: 53.92%
Epoch 3, Iter 24, Avg Loss: 1.3168
Epoch 3, Iter 48, Avg Loss: 1.2842
Epoch 3, Iter 72, Avg Loss: 1.2765
Epoch 3, Iter 96, Avg Loss: 1.2271
Validation Epoch 3 Accuracy: 54.50%
Training Epoch 3 Accuracy: 55.83%
Epoch 4, Iter 24, Avg Loss: 1.2070
Epoch 4, Iter 48, Avg Loss: 1.2105
Epoch 4, Iter 72, Avg Loss: 1.1912
Epoch 4, Iter 96, Avg Loss: 1.1722
Validation Epoch 4 Accuracy: 58.80%
Training Epoch 4 Accuracy: 60.43%
Epoch 5, Iter 24, Avg Loss: 1.1652
Epoch 5, Iter 48, Avg Loss: 1.1569
Epoch 5, Iter

[I 2025-12-01 19:11:00,493] Trial 1 finished with value: 0.607 and parameters: {'learn_rate': 0.00037308596728276506, 'epochs': 6}. Best is trial 1 with value: 0.607.


...

Training Epoch 12 Accuracy: 68.70%
Early stopping triggered after 12 epochs.
Epoch 1, Iter 24, Avg Loss: 2.1383
Epoch 1, Iter 48, Avg Loss: 1.8734
Epoch 1, Iter 72, Avg Loss: 1.7378
Epoch 1, Iter 96, Avg Loss: 1.6580
Validation Epoch 1 Accuracy: 41.30%
Training Epoch 1 Accuracy: 42.81%
Epoch 2, Iter 24, Avg Loss: 1.5827
Epoch 2, Iter 48, Avg Loss: 1.5437
Epoch 2, Iter 72, Avg Loss: 1.5216
Epoch 2, Iter 96, Avg Loss: 1.4829
Validation Epoch 2 Accuracy: 47.10%
Training Epoch 2 Accuracy: 49.18%
Epoch 3, Iter 24, Avg Loss: 1.4530
Epoch 3, Iter 48, Avg Loss: 1.4225
Epoch 3, Iter 72, Avg Loss: 1.3948
Epoch 3, Iter 96, Avg Loss: 1.3866
Validation Epoch 3 Accuracy: 49.80%
Training Epoch 3 Accuracy: 53.18%
Epoch 4, Iter 24, Avg Loss: 1.3575
Epoch 4, Iter 48, Avg Loss: 1.3307
Epoch 4, Iter 72, Avg Loss: 1.3204
Epoch 4, Iter 96, Avg Loss: 1.3068
Validation Epoch 4 Accuracy: 53.80%
Training Epoch 4 Accuracy: 54.90%
Epoch 5, Iter 24, Avg Loss: 1.2750
Epoch 5, Iter 48, Avg Loss: 1.2623
Epoch 5, Iter

[I 2025-12-01 19:22:30,997] Trial 23 finished with value: 0.611 and parameters: {'learn_rate': 0.00018524748033396296, 'epochs': 16}. Best is trial 13 with value: 0.694.


Training Epoch 13 Accuracy: 64.14%
Early stopping triggered after 13 epochs.
Epoch 1, Iter 24, Avg Loss: 2.0590
Epoch 1, Iter 48, Avg Loss: 1.7077
Epoch 1, Iter 72, Avg Loss: 1.6097
Epoch 1, Iter 96, Avg Loss: 1.5514
Validation Epoch 1 Accuracy: 47.20%
Training Epoch 1 Accuracy: 47.57%
Epoch 2, Iter 24, Avg Loss: 1.4813
Epoch 2, Iter 48, Avg Loss: 1.4299
Epoch 2, Iter 72, Avg Loss: 1.3863
Epoch 2, Iter 96, Avg Loss: 1.3641
Validation Epoch 2 Accuracy: 49.20%
Training Epoch 2 Accuracy: 50.92%
Epoch 3, Iter 24, Avg Loss: 1.3043
Epoch 3, Iter 48, Avg Loss: 1.2914
Epoch 3, Iter 72, Avg Loss: 1.2300
Epoch 3, Iter 96, Avg Loss: 1.2215
Validation Epoch 3 Accuracy: 54.70%
Training Epoch 3 Accuracy: 56.90%
Epoch 4, Iter 24, Avg Loss: 1.1477
Epoch 4, Iter 48, Avg Loss: 1.1573
Epoch 4, Iter 72, Avg Loss: 1.1499
Epoch 4, Iter 96, Avg Loss: 1.1524
Validation Epoch 4 Accuracy: 60.30%
Training Epoch 4 Accuracy: 61.26%
Epoch 5, Iter 24, Avg Loss: 1.0956
Epoch 5, Iter 48, Avg Loss: 1.1068
Epoch 5, Iter

[I 2025-12-01 19:23:04,261] Trial 24 finished with value: 0.674 and parameters: {'learn_rate': 0.002037837734805069, 'epochs': 11}. Best is trial 13 with value: 0.694.


Training Epoch 11 Accuracy: 70.56%


## Visualizations

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
import os

sns.set_theme(context="talk", style="whitegrid")

FIGSIZE = {
    "full": (6.4, 3.6),
    "half": (3.1, 1.8),
    "tall": (5.0, 4.0),
    "wide": (7.0, 3.5),
}

plt.rcParams.update({
    # --- Figure layout ---
    "figure.figsize": FIGSIZE["full"],      # 16:9 aspect ratio (full width)
    "figure.dpi": 200,                 
    "savefig.dpi": 300,                

    # --- Font sizes ---
    "font.size": 10,                   # match Beamer base font
    "axes.titlesize": 11,
    "axes.labelsize": 10,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "legend.fontsize": 9,

    # --- Fonts ---
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Fira Sans"],
    "text.latex.preamble": r"""
        \usepackage{FiraSans}
        \usepackage{sfmath}
        \renewcommand*\familydefault{\sfdefault}
    """,

    # --- Axes and grid ---
    "axes.spines.top": False,
    "axes.spines.right": False,
    "axes.grid": True,
    "grid.linestyle": ":",
    "grid.alpha": 0.3,

    # --- Save options ---
    # "savefig.bbox": "tight",
    "savefig.pad_inches": 0.02,
})

In [56]:
def savefig(name, ext="pdf", imag_dir="images", transparent=True, dpi=300):
    """
    Save the current matplotlib figure in a clean, consistent format.

    Parameters
    ----------
    name : str
        Base filename (no extension).
    ext : str, optional
        File extension, e.g. 'pdf', 'png', or 'svg'. Default is 'pdf'.
    transparent : bool, optional
        Use transparent background (useful for LaTeX Beamer overlays). Default is True.
    dpi : int, optional
        Dots per inch for raster formats (e.g., PNG). Default is 300.
    """

    # Ensure output directory exists
    os.makedirs(imag_dir, exist_ok=True)
    
    path = os.path.join(imag_dir, f"{name}.{ext}")
    plt.savefig(path, dpi=dpi, bbox_inches="tight", transparent=transparent)

    plt.close()
    print(f"💾 Saved: {path}")

def plot_contour(df, filename, df_hidden):
    """
    df         → DataFrame der Suchmethode (z.B. df_grid / df_random / df_bayesian)
    filename   → Name für savefig()
    df_hidden  → vollständiger Hyperparameter-Grid (für Contour)
    """

    # build lr/epoch grid
    lr_vals = np.sort(df_hidden["lr"].unique())
    epoch_vals = np.sort(df_hidden["epochs"].unique())
    df_hidden_unique = df_hidden.drop_duplicates(subset=["epochs", "lr"])

    Z = (
        df_hidden_unique
        .pivot(index="epochs", columns="lr", values="val_acc")
        .reindex(index=epoch_vals, columns=lr_vals)
    )

    X, Y = np.meshgrid(lr_vals, epoch_vals)

    # Number of contour levels = reasonable (not N points!)
    n_levels = df_hidden.shape[0] // 10

    plt.figure(constrained_layout=True)

    # no white contour edges → linewidths=0
    cs = plt.contourf(
        X, Y, Z.values,
        levels=n_levels,
        cmap="viridis",
    )
    plt.xscale("log")
    plt.xlabel("Lernrate")
    plt.ylabel("Epochen")

    # Colorbar formatting
    cbar = plt.colorbar(cs, label="Accuracy")
    cbar.ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))

    # Epoch axis integer format
    plt.gca().yaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f'))

    # Search points
    plt.scatter(df["lr"], df["epochs"], color="black", label="Suchpunkte")

    # Best point
    # best = df.loc[df["val_acc"].idxmax()]
    # plt.scatter(
    #     best["lr"], best["epochs"],
    #     color="red",
    #     s=60,
    #     label=f"Bestes Ergebnis = {best['val_acc']:.2f}"
    # )

    plt.legend()
    savefig(filename)
    plt.show()

In [57]:
df_hidden = pd.read_csv("results/hidden.csv", index_col=0)
df_grid = pd.read_csv("results/grid.csv", index_col=0)
df_random = pd.read_csv("results/random.csv", index_col=0)
df_bayesian = pd.read_csv("results/bayesian.csv", index_col=0)

In [58]:
plot_contour(df_grid, "contour_grid", df_hidden)
plot_contour(df_random, "contour_random", df_hidden)
plot_contour(df_bayesian, "contour_bayesian", df_hidden)

💾 Saved: images\contour_grid.pdf
💾 Saved: images\contour_random.pdf
💾 Saved: images\contour_bayesian.pdf


In [59]:
# Prepare combined dataframe
df_all = pd.concat([df_grid, df_random, df_bayesian], ignore_index=True)

# Compute global max and band
global_max = df_all["val_acc"].max()
LOWER_BOUND = 0.9
lower_band = global_max * LOWER_BOUND

# Create FacetGrid with columns per method
# TODO: improve size and aspect ratio for 16:9 beamer.
g = sns.FacetGrid(
    df_all,
    col="method",
    col_wrap=3,
    height=4,
    aspect=1.2,
    sharex=True,
    sharey=True
)

# Scatter plot
g.map_dataframe(
    sns.scatterplot,
    x="trial",
    y="val_acc",
    # s=25,
    # alpha=0.7
)

# Add rolling max line + hspan
for ax, method in zip(g.axes.flat, df_all["method"].unique()):
    df_m = df_all[df_all["method"] == method].sort_values("trial")
    
    # Rolling max
    ax.plot(
        df_m["trial"], 
        df_m["val_acc"].cummax(), 
        color="orange", 
        linestyle="--",
        label="Aktueller Bestwert")

    # top band
    ax.axhspan(lower_band, global_max, 
            color="grey", 
            alpha=0.15, 
            label=f"Top {int(np.round((1-LOWER_BOUND)*100))}% Band"
    )
    
    # Labels
    ax.set_title(method)
    ax.set_xlabel("Trial")
    ax.set_ylabel("Accuracy")

# handles, labels = ax.get_legend_handles_labels()
# g.fig.legend(
#     handles,
#     labels,
#     loc="lower center",
#     ncol=3,
#     frameon=False
# )

plt.tight_layout()
savefig("comparison")
plt.show()

💾 Saved: images\comparison.pdf
